# Inference with sCellTransformer

[![Open All Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/instadeepai/nucleotide-transformer/blob/main/examples/inference_segment_nt.ipynb)

## Installation and imports

In [2]:
import os

try:
    import nucleotide_transformer
except:
    !pip install git+https://github.com/instadeepai/nucleotide-transformer@main |tail -n 1
    import nucleotide_transformer

if "COLAB_TPU_ADDR" in os.environ:
    from jax.tools import colab_tpu

    colab_tpu.setup_tpu()

In [ ]:
import haiku as hk
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
from nucleotide_transformer.sCellTransformer.model import build_long_range_nt_fn
from nucleotide_transformer.sCellTransformer.params import download_ckpt

jax.config.update("jax_platform_name", "cpu")

backend = "cpu"
devices = jax.devices(backend)
num_devices = len(devices)
print(f"Devices found: {devices}")

Devices found: [CpuDevice(id=0)]


In [ ]:
parameters, config = download_ckpt()

forward_fn = build_long_range_nt_fn(model_config)

In [ ]:
forward_fn = hk.transform_with_state(forward_fn)
apply_fn = jax.pmap(forward_fn.apply, devices=devices, donate_argnums=(0,))

# Put required quantities for the inference on the devices. This step is not
# reproduced in the second inference since the quantities will already be loaded
# on the devices !
random_key = jax.random.PRNGKey(seed=0)
keys = jax.device_put_replicated(random_key, devices=devices)
parameters = jax.device_put_replicated(parameters, devices=devices)

# Data

In [ ]:
num_cells = config.num_cells
dummy_gene_expressions = np.random.randint(0, 5, size=(1, 19968 * num_cells))

## Infer on batch

In [ ]:
# Infer
outs = apply_fn(parameters, keys, dummy_gene_expressions) 

# Obtain the logits over the genomic features
logits = outs["logits"]
probabilities = np.asarray(jax.nn.softmax(logits[:, :, :5], axis=-1))[...,-1]

/opt/miniconda3/envs/genomics-research-env/lib/python3.11/site-packages/jax/_src/interpreters/mlir.py:914: UserWarning: Some donated buffers were not usable: ShapedArray(float32[608,1]), ShapedArray(float32[5,512,608]), ShapedArray(float32[736,1]), ShapedArray(float32[5,608,736]), ShapedArray(float32[896,1]), ShapedArray(float32[5,736,896]), ShapedArray(float32[1056,1]), ShapedArray(float32[5,896,1056]), ShapedArray(float32[1280,1]), ShapedArray(float32[5,1056,1280]), ShapedArray(float32[1536,1]), ShapedArray(float32[5,1280,1536]), ShapedArray(float32[1920,1]), ShapedArray(float32[1,1536,1920]), ShapedArray(float32[7611]), ShapedArray(float32[1920,7611]), ShapedArray(float32[512,1]), ShapedArray(float32[15,4,512]), ShapedArray(float32[1,8,1,64]), ShapedArray(float32[1,8,1,64]), ShapedArray(float32[1536,512]), ShapedArray(float32[1536]), ShapedArray(float32[1536,1536]), ShapedArray(float32[1536,512]), ShapedArray(float32[32,512]), ShapedArray(float32[1536,1536]), ShapedArray(float32[153

In [11]:
probabilities.shape

(1, 1, 196608, 14)